# Создание обработанного датасета

## Импорт необходимых библиотек

> Ячейки с установкой библиотек были приведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
from langdetect import detect  # для создания выборок сообщений на русском языке

In [3]:
import emoji
import regex
import re

In [4]:
from copy import deepcopy

## Первичная подготовка данных

### Чтение файлов подготовленных вручную

`data/dataset.json` – `.json` файл структуры `[{"text": "blablabla", "label": 0}]`, где ключ `text` представляет текстовое значение сообщения, а ключ `label` – класс сообщения (0: безопасное, 1: опасное).  
Сообщения были собраны из самого чата "ЧАТ_СТАНКИН Абитуриент".

In [5]:
df_manual = pd.read_json('../data/dataset.json')

In [6]:
df_manual.head()

,text,label
0,Добрый день! Отличается ли перечень необходимы...,0
1,Узбекистан. Рассматриваются обе формы,0
2,"Здравствуйте, а как проходит поступление после...",0
3,Спасибо большое за ответ!,0
4,"Здравствуйте, а когда будет день открытых двер...",0


`data/dataset_discord.json` – `.json` файл структуры `[{"text": "blablabla", "label": 0}]`, где ключ `text` представляет текстовое значение сообщения, а ключ `label` – класс сообщения (~~0: безопасное~~, 1: опасное).  
Сообщения были собраны из чатов Discord* сервера.  

> _Discord* – запрещенная социальная сеть в Российской Федерации._

In [7]:
df_discord = pd.read_json('../data/dataset_discord.json')

In [8]:
df_discord.head()

,text,label
0,# _**Лучший спам-бот!!! **_\n**залетай к нам н...,1
1,Лучший кряк нурсултана! https://wdfiles.ru/1tf...,1
2,https://oxy.name/d/pddi я заболел раокм гамна ...,1
3,Забирайте S.T.A.L.K.E.R 2 предзаказ на свой St...,1
4,https://only-fans.uk/morphe_ya\nOnlyFans UK - ...,1


### Чтение файлов, предоставленных сторонними ресурсами

#### ~~Telegram Kaggle Dataset~~

> Так как этот датасет нам не нужен (читайте далее) – ячейки были переведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

`data/dataset_telegram_kaggle.csv` – `.csv` (Comma-Separated Values) файл с колонками `text_type` и `text`, где `text_type` принимает значения `spam` или `ham`.  
Файл был получен с [Kaggle](https://www.kaggle.com/datasets/mexwell/telegram-spam-or-ham)

Приведение таблицы (датафрейма) к виду `['text', 'label']`

Так как в целевом чате используется преимущественно русский язык – образцы безопасных сообщений на иностранных языках нам не нужны. Выберем только те, что помечены `label = 1`

Выберем сообщения только на русском языке с помощью библиотеки `langdetect`

К сожалению, этот файл ценности для нас не имеет, так как в нём не оказалось спам-сообщений на русском языке

#### Umputun's tg-spam Dataset

`data/spam_samples_umputun.txt` – `.txt` файл с набором спам-сообщений.  
Файл был получен с [umputun/tg-spam GitHub](https://github.com/umputun/tg-spam/tree/master/data)

In [9]:
spam_tg_umputun = open('../data/spam_samples_umputun.txt', 'r', encoding='utf-8').readlines()
for i in range(len(spam_tg_umputun)):
    spam_tg_umputun[i] = spam_tg_umputun[i].strip().rstrip()

In [10]:
spam_tg_umputun

["Use this link and register your account now start earning in cryptocurrency and testify to others Https://accetgrowth.ltd Use this link and register your account now start earning in cryptocurrency and testify to others https://tinyurl.com/yrd86r5b Your investment is safe and secure with this transparent and legit platform https://tinyurl.com/yrd86r5b Don't miss this opportunity to earn with this platform and be financially stable click on the link to join https://urlz.fr/orYP Trusted and legit investment platform no hidden charges https://tinyurl.com/yrd86r5b",
 'Globalstockoptions is one of the best trade broker trade platform I have been trading with them more than 9 months and I have got withdrawal ib commission more than $350000, I have more than 250 plus clients today those trading with Globalstockoptions and best part no one is calling me for any issues or not even for any update. I am so grateful and thankful to Globalstockoption, finally we have broker who do not have issues

Приведение таблицы (датафрейма) к виду `['text', 'label']`

In [11]:
df_tg_umputun = pd.DataFrame()
df_tg_umputun['text'] = spam_tg_umputun
df_tg_umputun['label'] = [1] * len(spam_tg_umputun)

In [12]:
df_tg_umputun.head()

,text,label
0,Use this link and register your account now st...,1
1,Globalstockoptions is one of the best trade br...,1
2,Now I believe in what people say about you is ...,1
3,https://t.me/+jS6cJeri0ug5MDkx This is very ...,1
4,Investing in a genuine company with great inve...,1


#### LOLS Bot Data 2024

`data/parsed_lols_2024.txt` – `.txt` файл с набором спам-сообщений.  
Файл был получен с [lols.bot](https://lols.bot/2024/)

In [13]:
parsed_lols_2024 = open('../data/parsed_lols_2024.txt', 'r', encoding='utf-8').readlines()

In [14]:
for i in range(len(parsed_lols_2024)):
    parsed_lols_2024[i] = parsed_lols_2024[i].replace('<br/>', '', 1).strip().replace('<br/>', '\n')

In [15]:
len(parsed_lols_2024)

34383

In [16]:
df_lols_2024 = pd.DataFrame()
df_lols_2024['text'] = parsed_lols_2024
df_lols_2024['label'] = [1] * len(parsed_lols_2024)

In [17]:
df_lols_2024.head()

,text,label
0,Нужны срочно ребятa нa офис 8000 4-6чaсов сроч...,1
1,Нужeн пoмoщник нa cклaд зп в день от 3000pyб п...,1
2,🔤🔤🔤🔤🔤🔤<br><br> НЕТ 18 ЛЕТ — НЕТ ДЕНЕГ <br...,1
3,☀️ Шиshᴋи<br>🌈 МĒF\n🔥 АЛЬФĀ PVР\n⭐️ ГĀШ\n🧉 Ķok...,1
4,✅✅✅✅✅<br>\n💲 Возьму в аренду ваш 🛒АВИТО🛒 аккау...,1


### Объединение подходящих файлов

In [18]:
df = pd.concat([df_manual, df_tg_umputun, df_discord, df_lols_2024])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61361 entries, 0 to 34382
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    61361 non-null  object
 1   label   61361 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [20]:
df.head()

,text,label
0,Добрый день! Отличается ли перечень необходимы...,0
1,Узбекистан. Рассматриваются обе формы,0
2,"Здравствуйте, а как проходит поступление после...",0
3,Спасибо большое за ответ!,0
4,"Здравствуйте, а когда будет день открытых двер...",0


## Обработка данных

### Создание новых столбцов с числовыми данными

В спам сообщениях очень часто используются эмодзи, ссылки, теги (`@`), много новых строк (`\n`) и лишних пробелов, так что их количество в сообщении может играть большую роль в классификации сообщений

#### Функции для подготовки текста и дополнительных числовых данных

> Эти же функции находятся в `../utils/preprocessing.py`.

In [21]:
def count_emojis(text: str) -> int:
    """
    Считает количество эмодзи в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных эмодзи.
    """
    # Разбиваем текст на "графемные кластеры" (учитывая составные эмодзи)
    data = regex.findall(r'\X', text)
    # Считаем эмодзи
    emoji_counter = sum(emoji.is_emoji(word) for word in data)
    return emoji_counter

In [22]:
def count_newlines(text: str) -> int:
    """
    Считает количество символов новой строки (\n) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество символов новой строки.
    """
    # Подсчёт символов новой строки
    newline_count = text.count('\n')
    return newline_count

In [23]:
def count_whitespaces(text: str) -> int:
    """
    Считает количество последовательных пробелов (два или более) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных последовательных пробелов.
    """
    # Считаем последовательности из двух и более пробелов
    extra_spaces_count = len(re.findall(r'\s{2,}', text))
    return extra_spaces_count

In [24]:
# Регулярное выражение для ссылок
url_pattern = r'(https?://[^\s]+|www\.[^\s]+|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?)'

In [25]:
def count_links(text: str) -> int:
    """
    Считает количество ссылок в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных ссылок.
    """
    # Найти все ссылки в тексте
    links = re.findall(url_pattern, text)
    return len(links)

In [26]:
def count_tags(text: str) -> int:
    """
    Считает количество упоминаний (тегов) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных тегов.
    """
    # Подсчёт символов "@" как маркеров тегов
    tag_count = text.count('@')
    return tag_count

Удаление пунктуации и эмодзи, приведение к нижнему регистру может снизить шум и сложность модели

In [27]:
def remove_emojis(text: str) -> str:
    """
    Удаляет все эмодзи из текста.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст без эмодзи.
    """
    # Разбиваем текст на "графемные кластеры"
    data = regex.findall(r'\X', text)
    # Удаляем все эмодзи
    text_without_emojis = ''.join(word for word in data if not emoji.is_emoji(word))
    return text_without_emojis

Ссылки и тэги без пунктуации превращаются в набор латинских символов, что будет мешать работе модели, но при этом их нельзя удалять из текстов, так как это важные маркеры. Поэтому их необходимо заменить на что-то вроде `[LINK]` и `[TAG]`

In [28]:
def replace_links(text: str) -> str:
    """
    Заменяет все ссылки в тексте на "[LINK]".

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст с заменёнными ссылками.
    """
    # Заменить ссылки на [LINK]
    text = re.sub(url_pattern, '[LINK]', text)
    return text

In [29]:
def replace_tags(text: str) -> str:
    """
    Заменяет все упоминания (теги) в тексте на "[TAG]".

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст с заменёнными тегами.
    """
    # Регулярное выражение для тегов
    tag_pattern = r'@[a-zA-Z0-9_]+'
    # Замена тегов на [TAG]
    text = re.sub(tag_pattern, '[TAG]', text)
    return text

In [30]:
def preprocess_text(
        text: str,
        lower: bool = True,
        remove_punctuation: bool = True,
        remove_emoji: bool = True,
        remove_whitespaces: bool = True,
        remove_links: bool = True,
        remove_tags: bool = True
) -> str:
    """
    Предобрабатывает текст, включая нормализацию регистра, удаление пунктуации, эмодзи, ссылок, тегов и пробелов.

    Параметры:
        text (str): Исходный текст.
        lower (bool): Приводить ли текст к нижнему регистру (по умолчанию True).
        remove_punctuation (bool): Удалять ли пунктуацию (по умолчанию True).
        remove_emoji (bool): Удалять ли эмодзи (по умолчанию True).
        remove_whitespaces (bool): Удалять ли лишние пробелы (по умолчанию True).
        remove_links (bool): Удалять ли ссылки (по умолчанию True).
        remove_tags (bool): Удалять ли теги (по умолчанию True).

    Возвращает:
        str: Обработанный текст.
    """
    # Приведение текста к нижнему регистру
    if lower:
        text = text.lower()
    # Замена ссылок на [LINK]
    if remove_links:
        text = replace_links(text)
    # Замена тегов на [TAG]
    if remove_tags:
        text = replace_tags(text)
    # Удаление пунктуации
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', '', text)  # Удаляет пунктуацию
        # Восстановление [LINK] после очистки пунктуации
        if remove_links:
            text = text.replace('LINK', '[LINK]')
        # Восстановление [TAG] после очистки пунктуации
        if remove_tags:
            text = text.replace('TAG', '[TAG]')
    # Удаление эмодзи
    if remove_emoji:
        text = remove_emojis(text)
    # Удаление лишних пробелов и символов новой строки
    if remove_whitespaces:
        text = re.sub(r'\s+', ' ', text).strip()
    return text

#### Подготовка текста и дополнительных числовых данных

In [31]:
df_preprocessed = deepcopy(df)

In [32]:
emojis_count = []
newlines_count = []
whitespaces_count = []
links_count = []
tags_count = []
for text in df['text']:
    emojis_count.append(count_emojis(text))
    newlines_count.append(count_newlines(text))
    whitespaces_count.append(count_whitespaces(text))
    links_count.append(count_links(text))
    tags_count.append(count_tags(text))

In [33]:
df_preprocessed['emojis'] = emojis_count
df_preprocessed['newlines'] = newlines_count
df_preprocessed['whitespaces'] = whitespaces_count
df_preprocessed['links'] = links_count
df_preprocessed['tags'] = tags_count

In [34]:
df_preprocessed.head()

,text,label,emojis,newlines,whitespaces,links,tags
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0
3,Спасибо большое за ответ!,0,0,0,0,0,0
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0


### Выявление корреляции между новыми параметрами и целевой переменной

Количество эмодзи действительно может влиять на результативность модели

In [35]:
df_preprocessed.sort_values('emojis', ascending=False)

,text,label,emojis,newlines,whitespaces,links,tags
5690,🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸🌸<br>🌸🌸🌸🌸🌸🍋🍋🌸🌸🌸🍋🍋🌸🌸🌸🌸🌸\n🌸🌸🌸🌸🍋🍋...,1,466,38,19,1,0
23178,СБЕРБАНК КАРТА 💳💳 АЛАМ<br>\n💵💵💵💵💵💵💵💵💵💵💵💵💵💵💵💵💵💵...,1,424,203,107,0,0
7717,🔥ЖИДКОСТИ🔥<br>😎БОЛЕЕ 5000 ДОВОЛЬНЫХ КЛИЕНТОВ😎\...,1,356,169,16,0,0
2261,🔺SALES MANAGER🔺<br>♦️МЕНЕДЖЕР ПО ПРОДАЖАМ♦️\n\...,1,319,82,30,0,0
32038,😤😤😤😤 <br>\n😤😤😤😤😤\n\n🟣БЕРЕМ РЕБЯТ С ОПЫТОМ И...,1,267,129,47,0,0
...,...,...,...,...,...,...,...
19783,.,0,0,0,0,0,0
19784,а о,0,0,0,0,0,0
19785,,0,0,0,0,0,0
19786,Требую звание «эксперт чата»,0,0,0,0,0,0


Количество символов новой строки, к сожалению, не вступает в закономерность

In [36]:
df_preprocessed.sort_values('newlines', ascending=False)

,text,label,emojis,newlines,whitespaces,links,tags
185,"ʙᥴᥱⲙ ᤋдρᥲʙᥴᴛʙуᥔᴛᥱ ,<br>ᥙ ϶ᴛ᧐ нᥲδ᧐ρ ʙ᧐ ɸ᧘уд\n""н...",1,81,236,82,0,0
18037,🔫🔫🔫 🔫🔫🔫🔫🔫<br>\n\n🥶🥶🥶 \n\n\n⭐𝗡𝗘𝗧𝗙𝗟𝗜𝗫 𝗣𝗥𝗜𝗩 𝗦𝗖...,1,82,229,125,0,3
22074,🔫🔫🔫 🔫🔫🔫🔫🔫<br>\n\n🥶🥶🥶 \n\n\n⭐𝗡𝗘𝗧𝗙𝗟𝗜𝗫 𝗣𝗥𝗜𝗩 𝗦𝗖...,1,82,229,125,0,4
19679,🔫🔫🔫 🔫🔫🔫🔫🔫<br>\n\n🥶🥶🥶 \n\n\n⭐𝗡𝗘𝗧𝗙𝗟𝗜𝗫 𝗣𝗥𝗜𝗩 𝗦𝗖...,1,82,229,125,0,3
20758,🔫🔫🔫 🔫🔫🔫🔫🔫<br>\n\n🥶🥶🥶 \n\n\n⭐𝗡𝗘𝗧𝗙𝗟𝗜𝗫 𝗣𝗥𝗜𝗩 𝗦𝗖...,1,82,229,125,0,3
...,...,...,...,...,...,...,...
19185,ты ещё пропустил отрубленные пальцы,0,0,0,0,0,0
19186,Тише-тише,0,0,0,0,0,0
19187,Вдруг его переманят,0,0,0,0,0,0
19189,На их сторону,0,0,0,0,0,0


Количество пробелов тоже может влиять на результативность модели

In [37]:
df_preprocessed.sort_values('whitespaces', ascending=False)

,text,label,emojis,newlines,whitespaces,links,tags
25359,Занимаемcя поставкой теxники Sаmsung из ОАЭ🇦🇪 ...,1,93,182,172,5,0
20399,🤖 П р о в е р ь д е в у ш е к ...,1,12,21,166,0,4
20254,И н т и м к и с п о м о щ ь...,1,12,15,162,0,4
18279,▬▬▬▬▬▬▬▬▬▬▬▬▬▬<br>\n🔰𝗢𝗧𝗧 𝗣𝗟𝗔𝗧𝗙𝗢𝗥𝗠𝗦 🔰\n\n➜ NET...,1,56,181,140,0,0
22690,💔🔠🔠🔠 🔠🔠🔠🔠🔠🔠🔠🔠 💔<br>\n⚡ NETFLIX CRACK \n⚡ NETF...,1,165,182,136,0,0
...,...,...,...,...,...,...,...
19458,ееееемае вы лютые,0,0,0,0,0,0
19459,для нее это был самый важный урок в нашей жизни,0,0,0,0,0,0
19460,"ой астрономию мы в 10 прошли, слава богу в 11 ...",0,0,0,0,0,0
19461,"а у нас её в 10 не было, зато она была в 11",0,0,0,0,0,0


Количество ссылок тоже может влиять на результативность модели

In [38]:
df_preprocessed.sort_values('links', ascending=False)

,text,label,emojis,newlines,whitespaces,links,tags
17536,"Completely FREE,crypto earning,NO deposit,NO i...",1,0,103,0,103,0
18499,"Completely FREE,crypto earning,NO deposit,NO i...",1,0,103,0,103,0
17369,"Completely FREE,crypto earning,NO deposit,NO i...",1,0,101,0,101,0
18711,"Completely FREE,crypto earning,NO deposit,NO i...",1,0,98,0,98,0
18710,https://bnbhs.com/ref/193822319366<br>https://...,1,0,87,0,87,0
...,...,...,...,...,...,...,...
20402,вот бы пойти в колледж на киберспортсмена,0,0,0,0,0,0
20403,Только в синергию😈,0,1,0,0,0,0
20404,"синергия топ, у меня там оба отца учились",0,0,0,0,0,0
20405,"Говорят там хорошее обучение, но вуз без лицен...",0,0,0,0,0,0


Количество тэгов тоже может влиять на результативность модели

In [39]:
df_preprocessed.sort_values('tags', ascending=False)

,text,label,emojis,newlines,whitespaces,links,tags
16379,✔️ SORGU PANELİ SATIN ALMAK İÇİN ✔️<br>\n\n\n@...,1,13,97,36,0,91
18692,💠SHARE YOUR LINK💠<br>\n𝟖𝟎 𝐠𝐫𝐨𝐮𝐩𝐬 𝐟𝐨𝐫 𝐥𝐢𝐧𝐤 𝐬𝐡𝐚𝐫...,1,2,91,93,2,81
21728,🌞🌞SHARE🌞🌞<br>\n𝟖𝟎 𝐠𝐫𝐨𝐮𝐩𝐬 𝐟𝐨𝐫 𝐥𝐢𝐧𝐤 𝐬𝐡𝐚𝐫𝐞.- \nht...,1,4,88,92,2,80
21729,🌟SHARE YOUR LINK🌟<br>\n𝟖𝟎 𝐠𝐫𝐨𝐮𝐩𝐬 𝐟𝐨𝐫 𝐥𝐢𝐧𝐤 𝐬𝐡𝐚𝐫...,1,2,88,92,2,80
21734,👑👑SHARE LINK👑👑<br>\n𝟖𝟎 𝐠𝐫𝐨𝐮𝐩𝐬 𝐟𝐨𝐫 𝐥𝐢𝐧𝐤 𝐬𝐡𝐚𝐫𝐞.-...,1,4,88,92,2,80
...,...,...,...,...,...,...,...
19853,Пойду поплачу,0,0,0,0,0,0
19854,целая единичка,0,0,0,0,0,0
19855,не надо плакать,0,0,0,0,0,0
19856,,0,0,0,0,0,0


In [40]:
texts = list(deepcopy(df['text']))
for text_i in range(len(texts)):
    temp = preprocess_text(texts[text_i])
    if not temp:
        temp = np.NaN
    texts[text_i] = temp

In [41]:
texts

['добрый день отличается ли перечень необходимых документов для иностранных граждан',
 'узбекистан рассматриваются обе формы',
 'здравствуйте а как проходит поступление после колледжа внутренний экзамен или только егэ и возможно ли всё ещё поступление сразу на второй курс',
 'спасибо большое за ответ',
 'здравствуйте а когда будет день открытых дверей и во сколько',
 'спасибо',
 'здравствуйте необходимо для посещения дня открытым дверей гдето регистрироваться',
 'добрый день да необходимо пройти быструю регистрацию на сайте [LINK]',
 'лучшая специальность жаль с моими баллами не пройти',
 'было бы неплохо если бы прикладывались ссылки для просмотра результатов экзаменов',
 'здравствуйте не смог найти списки поступающих на сайте ещё не опубликованы спасибо',
 'теперь увидел спасибо а как часто списки обновляются',
 'подскажите где найти ранжированные списки поступающих этого года по основному конкурсу',
 'вопрос к поступающим абитуриентам какая ориентировка на подачу заявления на соглас

In [42]:
len(texts)

61361

In [43]:
df_preprocessed['text_preprocessed'] = texts

In [44]:
df_preprocessed.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


In [45]:
df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61361 entries, 0 to 34382
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               61361 non-null  object
 1   label              61361 non-null  int64 
 2   emojis             61361 non-null  int64 
 3   newlines           61361 non-null  int64 
 4   whitespaces        61361 non-null  int64 
 5   links              61361 non-null  int64 
 6   tags               61361 non-null  int64 
 7   text_preprocessed  59393 non-null  object
dtypes: int64(6), object(2)
memory usage: 4.2+ MB


In [46]:
sum(df_preprocessed.isnull().sum(axis = 1))

1968

Чтобы проверить корреляцию – нужно оставить в таблице только числовые значения

In [47]:
df_corr = deepcopy(df_preprocessed)
del df_corr['text']
del df_corr['text_preprocessed']

Итоги корреляции:
- **Эмодзи**, **лишние пробелы**, **переносы строк** оказывают **умеренное влияние** на предсказание класса (`label`).
- **Ссылки** и **тэги** имеют **слабую корреляцию с целевой переменной**, но их использование может быть полезно в сочетании с другими признаками.

In [48]:
df_corr.corr()

,label,emojis,newlines,whitespaces,links,tags
label,1.000000,0.376377,0.429107,0.388612,0.154763,0.214522
emojis,0.376377,1.000000,0.773917,0.695045,0.350616,0.245231
newlines,0.429107,0.773917,1.000000,0.897296,0.453873,0.342778
whitespaces,0.388612,0.695045,0.897296,1.000000,0.368590,0.326096
links,0.154763,0.350616,0.453873,0.368590,1.000000,0.037617
tags,0.214522,0.245231,0.342778,0.326096,0.037617,1.000000


### Удаление лишних строк

In [49]:
df_preprocessed.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


Так как столбец `text` нам больше не нужен, а столбец `newlines_count` не несёт в себе полезности – их нужно удалить

In [50]:
df_preprocessed.dropna(subset=['text_preprocessed'], inplace=True)

In [51]:
df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59393 entries, 0 to 34382
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               59393 non-null  object
 1   label              59393 non-null  int64 
 2   emojis             59393 non-null  int64 
 3   newlines           59393 non-null  int64 
 4   whitespaces        59393 non-null  int64 
 5   links              59393 non-null  int64 
 6   tags               59393 non-null  int64 
 7   text_preprocessed  59393 non-null  object
dtypes: int64(6), object(2)
memory usage: 4.1+ MB


In [52]:
count_zero = df_preprocessed[df_preprocessed['label'] == 0].shape[0]

In [53]:
count_zero

24739

In [54]:
count_one = df_preprocessed[df_preprocessed['label'] == 1].shape[0]

In [55]:
count_one

34654

Итого мы получили 59393 строк, из которых:
- 24739 чистых сообщений
- 34654 сообщений, помеченных как спам

## Сохранение данных

In [56]:
df_preprocessed.to_csv('../data/preprocessed.csv')